# model 2(population dynamics)


* $E(t)$: *E. coli* population
* $C(t)$: *C. glutamicum* (Coryne) population

$$
\frac{dE}{dt} = \mu_E \cdot a_E  \cdot E - \delta_E \cdot E
$$

$$
\frac{dC}{dt} = \mu_C \cdot a_C  \cdot C \cdot \left(1 - \theta \cdot C\right) 
$$

In [ ]:
import numpy as np
import pandas as pd
from mxlpy import Model, scan, plot, Simulator

## Define model

In [ ]:
from mxlpy.types import unwrap


def v0(mu_e: float, a_e: float, e: float) -> float:
    return mu_e * a_e * e


def v1(delta_e: float, e: float) -> float:
    return delta_e * e


def v2(mu_c: float, a_c: float, c: float) -> float:
    return mu_c * a_c * c


def v3(v2: float, theta: float, c: float) -> float:
    return v2 * theta * c


def get_model_2() -> Model:
    return (
        Model()
        .add_variables({"E": 5.0, "C": 5.0})
        .add_parameters(
            {
                "mu_e": 0.4,
                "mu_c": 0.3,
                "a_e": 0.1,
                "a_c": 0.1,
                "delta_e": 0.1,
                "theta": 0.001,
            }
        )
        .add_reaction("v0", v0, args=["mu_e", "a_e", "E"], stoichiometry={"E": 1})
        .add_reaction("v1", v1, args=["delta_e", "E"], stoichiometry={"E": -1})
        .add_reaction("v2", v2, args=["mu_c", "a_c", "C"], stoichiometry={"C": 1})
        .add_reaction("v3", v3, args=["v2", "theta", "C"], stoichiometry={"C": -1})
    )


scenarios = pd.DataFrame(
    [
        {"a_e": 1.0, "a_c": 0.3},
        {"a_e": 0.9, "a_c": 0.4},
        {"a_e": 0.8, "a_c": 0.5},
        {"a_e": 0.7, "a_c": 0.6},
        {"a_e": 0.4, "a_c": 0.8},
        {"a_e": 0.3, "a_c": 1.0},
    ]
)


## Run scan (v1)

Here we use an explicit for loop to do the simulation.  
This should be easier to read for the first time, but has a few disadvantages

- the for loop is run sequentially, so you don't get any speed benefits from having more than 1 CPU core
- if you use `simulate` instead of `simulate_time_course` it's easy to get mismatching time points, which make comparisons harder
- you don't have access to previous simulation results (easy to fix though)
- more verbose



In [ ]:
fig, axs = plot.grid_layout(len(scenarios))
for ax, (_, pars) in zip(axs, scenarios.iterrows()):
    res = unwrap(
        Simulator(get_model_2())
        .update_parameters(pars.to_dict())
        .simulate_time_course(np.linspace(0, 14, 11))
        .get_result()
    )
    plot.lines(
        res.variables,
        xlabel="",
        ylabel="",
        ax=ax,
    )
    ax.set_title("a_E={}, a_C={}".format(*pars))
plot.grid_labels(axs, xlabel="Time / h", ylabel="Population")


## Run scan (v2)

To run this kind (and other kinds of scans for that matter), you can also use the functions supplied by the `scan` module.

All these routines run in parallel automatically, and offer the possibility to cache long calculations using the `cache` argument.

You can obtain the variables and fluxes with `.variables` and `.fluxes` respectively and check the parameters / initial conditions etc you put into the scan with `.to_scan`. 

You can also get aggregates over time or run (not super useful in your case using)

```python
res.get_agg_per_time("mean")
res.get_agg_per_run("mean")
```

Challenges here:

- You have to learn how to use a `pandas.MultiIndex` because the resulting dataframe will look something like this. But that's a nice thing to learn in general, it will pop up in all kinds of data analysis down the road
  

|              (n, time) |        E |       C |
|:-----------------------|---------:|--------:|
| (0, 0.0)               |  5       | 5       |
| (0, 1.4)               |  7.60981 | 5.66761 |
| (0, 2.8)               | 11.5818  | 6.42378 |
| (0, 4.199999999999999) | 17.6271  | 7.28009 |
| (0, 5.6)               | 26.8278  | 8.24962 |

In [ ]:
res = scan.time_course(
    get_model_2(),
    to_scan=scenarios,
    time_points=np.linspace(0, 14, 11),
)

fig, axs = plot.grid_layout(len(scenarios))
for ax, (i, df) in zip(axs, res.variables.groupby(level=0)):
    plot.lines(
        df.droplevel(0).rename(columns={"E": "E. coli", "C": "Coryne"}),
        ax=ax,
        xlabel="",
        ylabel="",
    )
    ax.set_title("a_E={}, a_C={}".format(*res.to_scan.loc[i]))
plot.grid_labels(axs, xlabel="Time / h", ylabel="Population")
